In [2]:
import pandas as pd
import json

sheets = pd.ExcelFile("C://Users//Zouheir//Angular//gestion-budget/Situation caisse 07-12 au 12-12-2020.xlsx").sheet_names

In [18]:
week_list = []
for i, sh in enumerate(sheets[:-1]):
    xls = pd.read_excel("C://Users//Zouheir//Angular//gestion-budget/Situation caisse 07-12 au 12-12-2020.xlsx", sheet_name=sh, header=6)
    rename_dict = {'Budget': 'budget_type',
     'Référence marché': 'market_ref',
     'Rubrique': 'rubric',
     'Compte comptable ': 'accountant_no',
     'Opération': 'operation',
     'Private Security Atlanta': 'psa',
     'Najd': 'najd',
     "Sté d'Aménagement": 'planning_company',
     'XY CONSEIL': 'xy_advisor',
     'Direction des Etudes (Holding)': 'holding',
     'TOTAL': 'total_sum'}
    xls.rename(columns=rename_dict, inplace=True)
    xls['budget_type'] = pd.Series(xls['budget_type']).fillna(method='ffill')
    xls['rubric'] = pd.Series(xls['rubric']).fillna(method='ffill')
    xls['accountant_no'] = pd.Series(xls['accountant_no']).fillna(method='ffill')
    xls['date'] = f"{i+21}/12/2020"
    # xls.drop(labels=['Unnamed: 11', 'Unnamed: 0'], axis='columns', inplace=True)
    # xls.dropna(subset=['MO'], axis='index', inplace=True)
    # xls.set_index('Unnamed: 0', inplace=True)
    xls = xls.apply(lambda x: x.fillna(0) if x.dtype.kind in 'biufc' else x.fillna(''))
    break

In [19]:
xls.head(3)

,budget_type,market_ref,rubric,accountant_no,operation,psa,najd,planning_company,xy_advisor,holding,total_sum,date
0,Fonctionnement,,Achats non stockés de matières et de fournitures,61227,Fournitures de bureau,0,0.0,0.0,0.0,0.0,0.0,21/12/2020
1,Fonctionnement,,Achats non stockés de matières et de fournitures,61251,Achats de fournitures non stockables (Eau),0,0.0,0.0,0.0,0.0,0.0,21/12/2020
2,Fonctionnement,,Achats non stockés de matières et de fournitures,61251,Achats de fournitures non stockables (Electric...,0,0.0,0.0,0.0,0.0,0.0,21/12/2020


In [70]:
def prepare(x):
    y = [e for e in x]
    y[2] = str(x[2])
    return tuple(y[:-1])

idx = xls.set_index(['budget_type','rubric', 'accountant_no', 'operation']).stack().index
idx_cols = [prepare(x) for x in list(idx)]
idx_cols

[('Fonctionnement',
  'Achats non stockés de matières et de fournitures',
  '61227',
  'Fournitures de bureau'),
 ('Fonctionnement',
  'Achats non stockés de matières et de fournitures',
  '61227',
  'Fournitures de bureau'),
 ('Fonctionnement',
  'Achats non stockés de matières et de fournitures',
  '61227',
  'Fournitures de bureau'),
 ('Fonctionnement',
  'Achats non stockés de matières et de fournitures',
  '61227',
  'Fournitures de bureau'),
 ('Fonctionnement',
  'Achats non stockés de matières et de fournitures',
  '61227',
  'Fournitures de bureau'),
 ('Fonctionnement',
  'Achats non stockés de matières et de fournitures',
  '61227',
  'Fournitures de bureau'),
 ('Fonctionnement',
  'Achats non stockés de matières et de fournitures',
  '61227',
  'Fournitures de bureau'),
 ('Fonctionnement',
  'Achats non stockés de matières et de fournitures',
  '61227',
  'Fournitures de bureau'),
 ('Fonctionnement',
  'Achats non stockés de matières et de fournitures',
  '61251',
  'Achats d

In [71]:
lim = -1


budget_types = dict((b, {"rubrics": list(set(x[1].strip() for x in idx_cols[:lim] if x[0] == b))}) for b in {x[0].strip() for x in idx_cols[:lim]})
rubrics = dict((b, {"account_no": list(set(x[2].strip() for x in idx_cols[:lim] if x[1] == b))}) for b in {x[1].strip() for x in idx_cols[:lim]})
account_nums = dict((b, {"operation": list(set(x[3].strip() for x in idx_cols[:lim] if x[2] == b))}) for b in {x[2].strip() for x in idx_cols[:lim]})


hierarchy = dict(budget_types=budget_types, rubrics=rubrics, account_nums=account_nums)
with open('hierarchy.json', 'w+') as f:
    json.dump(hierarchy, f)

In [92]:
for k, v in hierarchy['account_nums'].items():
    print(f'"{k}": {v},')

"2351": {'operation': ['Mobilier de bureau']},
"61462": {'operation': ['Dons']},
"61428": {'operation': ['Autres transports (Frais de correspondances)']},
"61713": {'operation': ['Indemnités et avantages divers']},
"61321": {'operation': ['Redevances des crédit-bail -mobilier', 'Redevances des crédit-bail - matériel']},
"61673": {'operation': ['Taxes sur les véhicules']},
"61745": {'operation': ['Assurances A.T']},
"61251": {'operation': ['Achats de fournitures non stockables (Electricité)', 'Achats de fournitures non stockables (Carburant)', 'Achats de fournitures non stockables (Eau)']},
"Compte comptable": {'operation': ['00/00/0000', '00/00/01', '00/00/000', 'Date', 'Solde final au 07/12/2020', 'Solde reporté au 07/12/2020', 'Total dotations au 07/12/2020', '00/00/00', 'Solde initial au 07/12/2020']},
"61331": {'operation': ['Entretien et réparations des biens immobiliers']},
"61431": {'operation': ['Voyages et déplacements']},
"3456": {'operation': ['Etat - Crédit de T.V.A. suivan